In [5]:
import cvxpy as cp 
import numpy as np

/home/samye/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def subproblem(x_s, meta_parameter, scenario_s, bm_s = None):
    n = meta_parameter['number of agents']
    K = meta_parameter['number of resources']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    
    alpha = 2*K
    reallo_cost = 0.01
    
    w_s = sum(evaluation(scenario_s['xi_1'][i]*x_s[i], alpha) for i in range(n))
    
    scenario_extended = {}

    for t in range(T):
        scenario_extended[t] = [scenario_s[t]['xi_2'][i]*(1-scenario_s['xi_1'][i])\
             for i in range(n)]
    

    prob_extended_list = [ scenario_s[j]['cond_prob'] for j in range(T)]

    z = cp.Variable((n, n))
    y = cp.Variable(n)
    

    sp = gp.Model("subproblem")
    
    z = sp.addVars(n,n, vtype=GRB.CONTINUOUS, name="z", lb=list(np.zeros((n,n)))) #reallocate
    y = sp.addVars(n,   vtype=GRB.CONTINUOUS, name="y", lb=list(np.zeros(n))) #new resources

    
    # conditional probability of extended stage, conditioning on S
    extended_obj = {}
    for t in range(T):
        extended_obj[t] = qsum(4*((scenario_extended[t][i]*y[i]/alpha) - \
                                    (scenario_extended[t][i]*y[i]/alpha)**2)  for i in range(n)) 
    extended_obj_exp = qsum(extended_obj[t]*prob_extended_list[t] for t in range(T))

    obj = cp.Maximize(w_s + extended_obj_exp  - reallo_cost*z.sum())
    sp = cp.Problem(obj, constraints)
    sp.setObjective( obj, GRB.MAXIMIZE)

    reallocate = sp.addConstrs( (y[i] == (1-scenario_s['xi_1'][i])*x_s[i] +\
                                     z.sum('*',i)-z.sum(i,'*') for i in range(n)), \
                                    name = 'reallocate')

    sp.setParam('OutputFlag', 0)
    sp.setParam('QCPDual', 1)
    

    max_it = 1e2
    it = 0

 
    if bm_s is None:
        sp.optimize()
        # print(sp.status)
        if sp.status == GRB.OPTIMAL:
            
            feas = 1
            
            lmbd = [reallocate[i].Pi for i in range(n)]
            
            cut_subdiff = [lmbd[i]*(1-scenario_s['xi_1'][i]) + 4*(1-2*(scenario_s['xi_1'][i]/alpha)*x_s[i])\
                *(scenario_s['xi_1'][i]/alpha) for i in range(n)]
            
            obj_val = obj.getValue()
            val_t = [extended_obj[t].getValue() for t in range(T)]
            print(obj_val, cut_subdiff)
            return cut_subdiff, feas, obj_val, val_t
        else:
            raise Exception('risk neutral subproblem is infeasible')
    # if given a benchmark, then add in event cut 
    else:

        value_list_b = [ bm_s['r_s'] + bm_s[t]['r_t'] for t in range(T) ]
        # short_fall_b = {}
        # for t in range(T):
        #     short_fall_b[t] =  sum(bm_s[tt]['cond_prob']*max(value_list_b[t] - value_list_b[tt],0) \
        #                                                 for tt in range(T))
        shtf = sp.addVars(T, T, vtype=GRB.CONTINUOUS, name="shtf", lb=list(np.zeros((T,T))))
        shtf_def = sp.addConstrs( (shtf[t, tt] >= value_list_b[t] - w_s +\
             reallo_cost*z.sum()- extended_obj[tt] for t in range(T)\
                 for tt in range(T)), name = 'shtf_def')
        
        #Write order constraint as T ineq constr: E[(eta-X)+]<=E[(eta-Y)+], for eta=Y(\omega_1), ...
        
        #expected shortfall of benchmark: E[(eta_t - Y)+]
        exp_shtf_bm = [sum(prob_extended_list[tt] * max(0,value_list_b[t]-value_list_b[tt]) for tt in range(T)) for t in range(T)]
        
        prob_extended_edit = dict( [((t,tt),prob_extended_list[tt]) for tt in range(T) for t in range(T)] )
        
        ord_constr = sp.addConstrs( (shtf.prod(prob_extended_edit, t, '*') <= exp_shtf_bm[t] for t in range(T)), name = 'ord_constr' )
        sp.optimize()
        if sp.status == GRB.OPTIMAL:
            short_fall = {}
            for t in range(T):
                short_fall[t] = sum(bm_s[tt]['cond_prob']*max(value_list_b[t]-\
                    w_s-extended_obj[tt].getValue() ,0) for tt in range(T))
            # print(sp.status)

            lmbd = [reallocate[i].Pi for i in range(n)]
            
            cut_subdiff = [lmbd[i]*(1-scenario_s['xi_1'][i]) + 4*(1-2*(scenario_s['xi_1'][i]/alpha)*x_s[i])\
                *(scenario_s['xi_1'][i]/alpha) for i in range(n)]
            
            obj_val = obj.getValue()
            val_t = [extended_obj[t].getValue() for t in range(T)]

            return cut_subdiff, 1, obj_val, val_t

                
        elif (sp.status == GRB.INFEASIBLE) or (sp.status == GRB.INF_OR_UNBD):
            
            sp.remove(reallocate)
            
            u = sp.addVars(n, vtype=GRB.CONTINUOUS, name="u", lb = -GRB.INFINITY)
            u_abs = sp.addVars(n, vtype=GRB.CONTINUOUS, name="u_abs")
            
            reallocate_again = sp.addConstrs( (u[i] + y[i] == (1-scenario_s['xi_1'][i])*x[i] + z.sum('*',i)-z.sum(i,'*') for i in range(n)),\
                                                name = 'reallocate_again')
            
            abs_of_u = sp.addConstrs( (u[i] <= u_abs[i] for i in range(n)) , name = 'abs_of_u')
            abs_of_u_ = sp.addConstrs( (-u[i] <= u_abs[i] for i in range(n)) , name = 'abs_of_u_')
            
            obj_feas = qsum(u_abs)
            sp.setObjective(obj_feas, GRB.MINIMIZE)
            
            sp.optimize()
            if sp.status != GRB.OPTIMAL:
                raise Exception(f'feasible subproblem status code {sp.status}')

            cut_val = obj_feas.getValue()
            # cut_val = sp.ObjVal
            
            lmbd = [reallocate_again[i].Pi for i in range(n)]
            
            
            cut_subdiff = [lmbd[i]*(1-scenario_s['xi_1'][i]) +\
                    4*(1-2*(scenario_s['xi_1'][i]/alpha)*x[i])*(scenario_s['xi_1'][i]/alpha) for i in range(n)]
            
            
            return cut_subdiff, 0, cut_val, [-float('inf') for i in range(T)]